In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
torch.__version__

In [ ]:
train_data = pd.read_csv('/kaggle/input/house-destruction/training_data.csv')
train_data.head()

testing_data = pd.read_csv('/kaggle/input/house-destruction/testing_data.csv')
testing_data.head()

In [ ]:
if "Unnamed: 0" in train_data.columns:
    del train_data['Unnamed: 0']
    print("del")

print(train_data['land_surface_condition'])
train_data['land_surface_condition'] = train_data['land_surface_condition'].fillna(0.0).map({'n': 0, 'o': 1, 't': 2}).astype(int)
train_data['foundation_type'] = train_data['foundation_type'].fillna(0.0).map({'h': 0, 'i': 1, 'r': 2 , 'u': 3, 'w': 4}).astype(int)
train_data['roof_type'] = train_data['roof_type'].fillna(0.0).map({'n': 0, 'q': 1, 'x': 2}).astype(int)
train_data['ground_floor_type'] = train_data['ground_floor_type'].fillna(0.0).map({'f': 0, 'm': 1, 'v': 2, 'x': 3, 'z': 4}).astype(int)
train_data['other_floor_type'] = train_data['other_floor_type'].fillna(0.0).map({'j': 0, 'q': 1, 's': 2, 'x': 3 }).astype(int)
train_data['position'] = train_data['position'].fillna(0.0).map({'j': 0, 'o': 1, 's': 2, 't':3 }).astype(int)
train_data['plan_configuration'] = train_data['plan_configuration'].fillna(0.0).map({'a': 0, 'c': 1, 'd': 2, 'f': 3 ,'m': 4 ,'n': 5 ,'o': 6,'q': 7,'s': 8,'u': 9 }).astype(int)
train_data['legal_ownership_status'] = train_data['legal_ownership_status'].fillna(0.0).map({'a': 0, 'r': 1, 'v': 2, 'w': 3 }).astype(int)


gft=train_data['ground_floor_type']
(train_data=='v').sum()

In [ ]:
if "Unnamed: 0" in testing_data.columns:
    del testing_data['Unnamed: 0']
    print("del")

print(train_data['land_surface_condition'])
testing_data['land_surface_condition'] = testing_data['land_surface_condition'].fillna(0.0).map({'n': 0, 'o': 1, 't': 2}).astype(int)
testing_data['foundation_type'] = testing_data['foundation_type'].fillna(0.0).map({'h': 0, 'i': 1, 'r': 2 , 'u': 3, 'w': 4}).astype(int)
testing_data['roof_type'] = testing_data['roof_type'].fillna(0.0).map({'n': 0, 'q': 1, 'x': 2}).astype(int)
testing_data['ground_floor_type'] = testing_data['ground_floor_type'].fillna(0.0).map({'f': 0, 'm': 1, 'v': 2, 'x': 3, 'z': 4}).astype(int)
testing_data['other_floor_type'] = testing_data['other_floor_type'].fillna(0.0).map({'j': 0, 'q': 1, 's': 2, 'x': 3 }).astype(int)
testing_data['position'] = testing_data['position'].fillna(0.0).map({'j': 0, 'o': 1, 's': 2, 't':3 }).astype(int)
testing_data['plan_configuration'] = testing_data['plan_configuration'].fillna(0.0).map({'a': 0, 'c': 1, 'd': 2, 'f': 3 ,'m': 4 ,'n': 5 ,'o': 6,'q': 7,'s': 8,'u': 9 }).astype(int)
testing_data['legal_ownership_status'] = testing_data['legal_ownership_status'].fillna(0.0).map({'a': 0, 'r': 1, 'v': 2, 'w': 3 }).astype(int)


gft=testing_data['ground_floor_type']
(testing_data=='v').sum()



In [ ]:
import sklearn
sklearn.__version__

In [ ]:
# train_start = 0
# train_end   = 200000
# test_start  = train_end
# test_end    = train_data.shape[0]

X    = train_data.drop('Category', axis=1).values
t    = train_data['Category'].values

X_testing    = testing_data.values

X.shape
t.shape

In [ ]:
X = torch.tensor(X,dtype=torch.float32)
t = torch.tensor(t,dtype=torch.int64)

X_testing = torch.tensor(X_testing,dtype=torch.float32)



t.type

In [ ]:
dataset = torch.utils.data.TensorDataset(X,t)

len(dataset)
# type(dataset)
# dataset[0][0]

In [ ]:
n_train = int(len(dataset)*0.6)+2
n_val = int(len(dataset)*0.2)
n_test = int(len(dataset)*0.2)

n_train,n_val,n_test

In [ ]:
train,val,test = torch.utils.data.random_split(dataset,[n_train,n_val,n_test])
len(train),len(val),len(test)

In [ ]:
batch_size = 5000
train_loader = torch.utils.data.DataLoader(train,batch_size,shuffle=True,drop_last=True)
val_loader = torch.utils.data.DataLoader(val,batch_size,shuffle=True,drop_last=True)
test_loader = torch.utils.data.DataLoader(test,batch_size,shuffle=True,drop_last=True)

train_loader

In [ ]:

import torch.nn as nn
import torch.nn.functional as F
fc1 = nn.Linear(39,10)
fc2 = nn.Linear(10,2)

In [ ]:
torch.manual_seed(0)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(39,10)
        self.fc2 = nn.Linear(10,2)
    
    def forward(self,x):
        h = self.fc1(x)
        h = F.relu(h)
        h = self.fc2(h)
        return h

In [ ]:
net = Net()
optimizer = torch.optim.SGD(net.parameters(),lr=0.1)
optimizer

In [ ]:
torch.manual_seed(0)

In [ ]:
net.forward(x.unsqueeze(0))

In [ ]:
print(*net.parameters())

In [ ]:
batch = next(iter(train_loader))
batch

In [ ]:
x,t = batch
x.shape

In [ ]:
y = net(x)
y

In [ ]:
loss = F.cross_entropy(y,t)

In [ ]:
optimizer.zero_grad()
loss.backward()


In [ ]:
optimizer.step()

In [ ]:
print(net.fc1.weight)

In [ ]:
torch.cuda.is_available()

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
net.to(device)

In [ ]:
x = x.to(device)
y = y.to(device)

In [ ]:
max_epoch = 1

In [ ]:
torch.manual_seed(0)
net = Net()

In [ ]:
optimizer = torch.optim.SGD(net.parameters(),lr=0.1)
optimizer

In [ ]:
for epoch in range(max_epoch):

    for batch in train_loader:
        x,t = batch
#         x = x.to(device)
#         t = t.to(device)
        
        y= net(x)
        
        y_label = torch.argmax(y,dim = 1)
        accuracy = (y_label == t).sum() / len(t)
        print(f'accuracy: {accuracy:.2f}')
        
        loss = F.cross_entropy(y,t)
        
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()

In [ ]:
y = net(x)
y

In [ ]:
t 

In [ ]:
y_label = torch.argmax(y,dim =1)
y_label

In [ ]:
(y_label == t).sum()/len(t)

In [ ]:
with torch.no_grad():
    def calc_accuracy(data_loader):
    
        total = 0
        correct = 0.0

        for batch in data_loader:
            x,t = batch

            y=net(x)

            y_label = torch.argmax(y,dim=1)
            total += len(t)
            correct += (y_label == t).sum()

            accuracy = correct / total

            return accuracy

In [ ]:
calc_accuracy(val_loader)

In [ ]:
calc_accuracy(test_loader)

In [ ]:
y=net(X_testing)

y_label2 = torch.argmax(y,dim =1)
y_label2

In [ ]:
sub = pd.read_csv('/kaggle/input/house-destruction/submission_format.csv')
sub['Category'] = list(map(int, y_label2))
sub.to_csv('submission_format.csv', index=False)